# decorator

In [10]:
# 最基本的用法

# The decorator to make it bold
def makebold(fn):
    # The new function the decorator returns
    def wrapper():
        '''makebold wrapper
        '''
        return "<b>" + fn() + "</b>"
    return wrapper

# The decorator to make it italic
def makeitalic(fn):
    # The new function the decorator returns
    def wrapper():
        '''makeitalic wrapper
        '''
        return "<i>" + fn() + "</i>"
    return wrapper

@makebold
@makeitalic
def hello():
    return "hello"

@makeitalic
@makebold
def hello_again():
    return "hello"

print(hello())
print(hello_again())

# 错误的函数名和 doc
print(hello_again.__name__)
print(hello.__doc__)
print(hello_again.__doc__)

<b><i>hello</i></b>
<i><b>hello</b></i>
wrapper
makebold wrapper
        
makeitalic wrapper
        


In [18]:
# 处理函数参数

def decorator_passing_arguments(func):
    def wrapper_accepting_arguments(*args, **kwargs):
        print(args, type(args))
        print(kwargs, type(kwargs))
        if 'a' in kwargs:
            kwargs['a'] += 100
        return func(*args, **kwargs)
    return wrapper_accepting_arguments

@decorator_passing_arguments
def func(a, b):
    return a + b

print(func(0, 0))
print()
print(func(a=0, b=0))

(0, 0) <class 'tuple'>
{} <class 'dict'>
0

() <class 'tuple'>
{'b': 0, 'a': 0} <class 'dict'>
100


In [24]:
# 带参数的装饰器

def decorator_maker(name='wrapper'):

    print("I am executed only once: "
          "when you make me create a decorator.")

    def my_decorator(func):

        print("I am a decorator! I am executed only when you decorate a function.")

        def wrapper():
            print("I'm", name)
            return func()

        print("As the decorator, I return the wrapped function.")
        return wrapper

    print("As a decorator maker, I return a decorator")
    return my_decorator

# 实际上，带参数的装饰器只是在装饰的时候多执行了一次函数，该函数返回一个装饰器
dec = decorator_maker()

print("*" * 42)

@dec
def f():
    print("f()")

f()

print("#" * 42)

@decorator_maker(name="god")
def g():
    print("g()")

g()

I am executed only once: when you make me create a decorator.
As a decorator maker, I return a decorator
******************************************
I am a decorator! I am executed only when you decorate a function.
As the decorator, I return the wrapped function.
I'm wrapper
f()
##########################################
I am executed only once: when you make me create a decorator.
As a decorator maker, I return a decorator
I am a decorator! I am executed only when you decorate a function.
As the decorator, I return the wrapped function.
I'm god
g()


In [23]:
# 使用 functools

import functools

def bar(func):
    # We say that "wrapper", is wrapping "func"
    # and the magic begins
    @functools.wraps(func)
    def wrapper():
        '''wrapper
        '''
        print("bar")
        return func()
    return wrapper

@bar
def foo():
    '''foo
    '''
    print("foo")

print(foo.__name__)
print(foo.__doc__)

foo
foo
    


In [25]:
# 记录函数使用记录

def counter(func):
    """
    A decorator that counts and prints the number of times a function has been executed
    """
    def wrapper(*args, **kwargs):
        wrapper.count = wrapper.count + 1
        res = func(*args, **kwargs)
        return res
    wrapper.count = 0
    return wrapper

@counter
def func():
    print("func")
    

func()
func()

print(func.count)

func
func has been used: 1x
func
func has been used: 2x
2


In [ ]:
# staticmethod and classmethod

class StaticMethod(object):
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, objtype=None):
        return self.f
    
class ClassMethod(object):
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f

    def __get__(self, obj, klass=None):
        if klass is None:
            klass = type(obj)
        def newfunc(*args):
            return self.f(klass, *args)
        return newfunc